# Gewinner und Verlierer von Listenverbindungen

In [2]:
## import libraries
import pandas as pd
import numpy as np
from requests import get
from tqdm import tqdm
import json
import math
import copy

---

## Dateien importieren

In [3]:
## define links to bfs-ressources
urlDict = {
    'meta_data': 'https://www.bfs.admin.ch/bfsstatic/dam/assets/9386467/master',
    'nr_candidates': 'https://www.bfs.admin.ch/bfsstatic/dam/assets/9386465/master',
    'lists': 'https://www.bfs.admin.ch/bfsstatic/dam/assets/9386464/master'
}

In [4]:
## download data
m = json.loads(get(urlDict['meta_data']).content.decode('utf-8-sig'))
nr = json.loads(get(urlDict['nr_candidates']).content.decode('utf-8-sig'))
l = json.loads(get(urlDict['lists']).content.decode('utf-8-sig'))

In [5]:
## extract relevant data
nr_kandidierende = pd.DataFrame.from_dict(nr["kandidierende"])
basislisten = pd.DataFrame.from_dict(l["listen"])

In [6]:
## reformat metadata
meta_parteien = []
for partei in m['parteien']:
    meta_parteien.append([partei['partei_id'],
                          partei['partei_bezeichnung_kurz'][0]['text'],
                          partei['partei_bezeichnung_kurz'][1]['text'],
                          partei['partei_bezeichnung'][0]['text'],
                          partei['partei_bezeichnung'][1]['text']])
    
meta_parteien = pd.DataFrame(meta_parteien,
                             columns=['partei_id',
                                      'partei_bezeichnung_kurz_de',
                                      'partei_bezeichnung_kurz_fr',
                                      'partei_bezeichnung_de',
                                      'partei_bezeichnung_fr'])


In [7]:
## create random party_ids for "übrige"
for i, row in basislisten[(basislisten.partei_id == 35)].iterrows():
    basislisten.loc[i, 'partei_id'] = i + 35

In [8]:
## create deepcopy of list
listen = copy.deepcopy(basislisten)

---

## Funktionen für spätere Berechnungen definieren

In [31]:
## function to distribute mandates and remaining seats on given Level
def calc_dist(df, verteilungszahl, sitze): # df: level1 indicator, level2 indicator, number of votes
    result = []
    
    # save rows without level2
    for i, row in df[df.iloc[:,1] == " "].iterrows():
        result.append([row[0], row[2], True, False, 1])
    
    # save rows with level2 (each level2 summed up as 1 row)
    for lv in df[df.iloc[:,1] != " "].iloc[:,1].unique():
        result.append([lv, df[df.iloc[:,1] == lv].stimmen_liste.sum(), False, True, len(df[df.iloc[:,1] == lv].iloc[:,3].unique())])
        
    result_df = pd.DataFrame(result, columns=["identifier", "stimmen", "lvl1", "lvl2", "n_p"])
    result_df['stimmen'] = result_df['stimmen'].astype(float)
    result_df['sitze'] = result_df['stimmen'].apply(lambda x: math.floor(x / verteilungszahl)) # calculate seats
    
    # in case of only 1 remaining party: assing all seats to them
    if len(result_df) == 1:
        result_df['sitze'] = sitze
        
    result_df['reststimmen'] = result_df['stimmen'].apply(lambda x: (x % verteilungszahl))
    result_df['restmandat'] = 0
    
    # distribute remaining seat
    if result_df['sitze'].sum() < sitze:
        restmandate = sitze - result_df['sitze'].sum()
        
        while restmandate > 0:
            result_df['restmandat_verteiler'] = result_df['stimmen'] / (result_df['sitze'] + 1)
            idx = result_df.sort_values(["restmandat_verteiler"], ascending=False).index[0]
            result_df.loc[idx, "sitze"] += 1
            result_df.loc[idx, "restmandat"] += 1
            restmandate -= 1
    
    # return dataframe
    return (result_df[["identifier", "stimmen", "lvl1", "lvl2", "sitze", "reststimmen", "restmandat", "n_p"]])

In [32]:
## function to loop through all cantons and levels (Listen, Listenverbindungen, Unterlistenverbindungen) and assign seats
def calc_seats(listenverbindungen, print_output = False):
    lv_res = []
    
    # loop through cantons
    for kanton in listenverbindungen.kanton_nummer.unique():
        listen_kt = listenverbindungen[listenverbindungen.kanton_nummer == kanton] # subset to canton
        kanton_name = listen_kt.kanton_bezeichnung.unique()[0] # extract name of canton
        nr_sitze = listen_kt.anzahl_gewaehlte.sum() # total number of seats for canton
        
        if print_output:
            print("{}: {}".format(kanton_name, nr_sitze))
        
        verteilungszahl = math.ceil(listen_kt.stimmen_liste.sum() / (nr_sitze + 1)) # calc "Verteilungszahl" (number of votes to get a seat)
        # only keep necessary columns and add an empty one
        listen_kt = listen_kt[["anzahl_gewaehlte", "liste_bezeichnung", "liste_nummer_kanton", "liste_unterlistenverbindung", "liste_verbindung", "partei_id", "stimmen_liste"]]
        listen_kt["empty"] = " "
        
        if print_output:
            print("------------------------ {} ------------------------".format(kanton_name))
            
        # analyse 1. level (use calc_dist())
        ebene1 = calc_dist(listen_kt[["liste_nummer_kanton", "liste_verbindung", "stimmen_liste", "partei_id"]], verteilungszahl, nr_sitze)
        
        if print_output:
            print("Ebene 1:")
            print(ebene1)
        
        # loop through results which don't have second level links
        for i, row in ebene1[ebene1.lvl1 == True].iterrows():
            
            if row['sitze'] > 0:
                
                if len(meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == row['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de) > 0:
                    parteiname = meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == row['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de.values[0]
                else:
                    parteiname = "übrige"
                
                # append result to final list
                lv_res.append([kanton, kanton_name, listen_kt[listen_kt.liste_nummer_kanton == row['identifier']].partei_id.values[0], parteiname, row['sitze'],  row['restmandat'], row['reststimmen'], "Listenstimmen", "", listen_kt[listen_kt.liste_nummer_kanton == row['identifier']].liste_bezeichnung.values[0], 1])
        
        # loop through results which have second level links
        for i, row in ebene1[(ebene1.sitze > 0) & (ebene1.lvl2 == True)].iterrows():
            # calc new "Verteilungszahl" (number of votes to get a seat based on the number of seats assigned to this group)
            verteilungszahl = math.ceil(row['stimmen'] / (row['sitze'] + 1))
            # analyse 2. level (use calc_dist())
            ebene2 = calc_dist(listen_kt[listen_kt.liste_verbindung == row['identifier']][["liste_nummer_kanton", "liste_unterlistenverbindung", "stimmen_liste", "partei_id"]], verteilungszahl, row['sitze'])
            
            if print_output:
                print("Ebene 2:")
                print(ebene2)
                
            # loop through results which don't have third level links    
            for j, subrow in ebene2[ebene2.lvl1 == True].iterrows():
                
                if subrow['sitze'] > 0:
                    
                    if len(meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == subrow['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de) > 0:
                        parteiname = meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == subrow['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de.values[0]
                    else:
                        parteiname = "übrige"
                    
                    # append result to final list
                    lv_res.append([kanton, kanton_name, listen_kt[listen_kt.liste_nummer_kanton == subrow['identifier']].partei_id.values[0], parteiname, subrow['sitze'],  subrow['restmandat'], subrow['reststimmen'], "Listenverbindung", row['identifier'], listen_kt[listen_kt.liste_nummer_kanton == subrow['identifier']].liste_bezeichnung.values[0], row['n_p']])
            
            # loop through results which have third level links    
            for j, subrow in ebene2[(ebene2.sitze > 0) & (ebene2.lvl2 == True)].iterrows():
                
                # calc new "Verteilungszahl" (number of votes to get a seat based on the number of seats assigned to this group)
                verteilungszahl = math.ceil(subrow['stimmen'] / (subrow['sitze'] + 1))
                # analyse 3. level (use calc_dist())
                ebene3 = calc_dist(listen_kt[listen_kt.liste_unterlistenverbindung == subrow['identifier']][["liste_nummer_kanton", "empty", "stimmen_liste", "partei_id"]], verteilungszahl, subrow['sitze'])
                
                if print_output:
                    print("Ebene 3:")
                    print(ebene3)
                    
                for k, subsubrow in ebene3[ebene3.lvl1 == True].iterrows():
                    
                    if subsubrow['sitze'] > 0:
                        
                        if len(meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == subsubrow['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de) > 0:
                            parteiname = meta_parteien[meta_parteien.partei_id == (listen_kt[listen_kt.liste_nummer_kanton == subsubrow['identifier']].partei_id.values[0])].partei_bezeichnung_kurz_de.values[0]
                        else:
                            parteiname = "übrige"
                            
                        # append result to final list
                        lv_res.append([kanton, kanton_name, listen_kt[listen_kt.liste_nummer_kanton == subsubrow['identifier']].partei_id.values[0], parteiname, subsubrow['sitze'],  subsubrow['restmandat'], subsubrow['reststimmen'], "Unterlistenverbindung", row['identifier'], listen_kt[listen_kt.liste_nummer_kanton == subsubrow['identifier']].liste_bezeichnung.values[0], row['n_p']])
                        
        if print_output:
            print(" ")
            
    return pd.DataFrame(lv_res, columns=["kanton_nummer", "kanton_name", "partei_id", "parteiname", "sitze", "restmandat", "reststimmen", "ebene", "listenverbindung_kennzeichen", "liste_bezeichnung", "anzahl_parteien"])


In [90]:
# function to remove links to other lists of given party
def del_lvs(partyid, listenverbindungen):
    # create deep copy of dataframe
    del_listenverbindungen = copy.deepcopy(listenverbindungen)
    
    #loop through cantons
    for kt in del_listenverbindungen.kanton_nummer.unique():
        temp = del_listenverbindungen[del_listenverbindungen.kanton_nummer == kt] #create temporary subset of df (reduced to 1 canton)
        # get all list links where the party is part of
        del_list = temp[(temp.partei_id == partyid)][["liste_verbindung", "partei_id"]].groupby("liste_verbindung").count().reset_index()
        
        # loop through listenverbindungen to delete
        for i, row in del_list.iterrows():
            
            if row['liste_verbindung'] != " ":
                    
                for j, subrow in temp.iterrows():

                    # replace list indicator with X
                    if ((subrow['partei_id'] == partyid) & (subrow['liste_verbindung'] == row['liste_verbindung'])):
                        del_listenverbindungen.loc[j, 'liste_verbindung'] = "X"

                        if del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'] != " ":
                            del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'] = "X{}".format(del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'][1])

                        #del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'] = " "

                    # replace list indicator for other memberrs of the listenverbindung with Y
                    if ((subrow['partei_id'] != partyid) & (subrow['liste_verbindung'] == row['liste_verbindung'])):
                        del_listenverbindungen.loc[j, 'liste_verbindung'] = "Y"

                        #if del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'] != " ":
                            #del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'] = "Y{}".format(del_listenverbindungen.loc[j, 'liste_unterlistenverbindung'][1])
                                
    #return new information                            
    return del_listenverbindungen

---

## Szenario 0
#### Echte Resulate replizieren

In [36]:
# replicate real result based on 'https://www.bfs.admin.ch/bfsstatic/dam/assets/9386464/master' (listen) 
lv_res_df = calc_seats(listen, print_output=True)

Zürich: 35
------------------------ Zürich ------------------------
Ebene 1:
  identifier    stimmen   lvl1   lvl2  sitze  reststimmen  restmandat  n_p
0         26    27242.0   True  False      0      27242.0           0    1
1         12    45514.0   True  False      0      45514.0           0    1
2         22     9540.0   True  False      0       9540.0           0    1
3         16     1851.0   True  False      0       1851.0           0    1
4         15     3063.0   True  False      0       3063.0           0    1
5          D  3406885.0  False   True      8     163821.0           0    4
6          B  4974479.0  False   True     12     109883.0           0    6
7          A  4131563.0  False   True     10      77733.0           0    2
8          C  1993644.0  False   True      5     372112.0           1    1
Ebene 2:
  identifier    stimmen   lvl1   lvl2  sitze  reststimmen  restmandat  n_p
0          7   239376.0   True  False      0     239376.0           0    1
1         D3  

In [35]:
# look at seat ditribution
lv_res_df[["parteiname", "sitze"]].groupby("parteiname").sum()

,sitze
parteiname,
BDP,3
CVP,25
EDU,1
EVP,3
FDP,28
GLP,16
GPS,28
LPS,1
Lega,1


---

## Szenario 1
#### Sitzzuteilung ganz ohne Listenverbindungen

In [9]:
## sum up votes by party and canton (therefore: ignore "Listenverbindungen")
parteien = listen[["kanton_bezeichnung", "kanton_nummer", "partei_id", "anzahl_gewaehlte", "stimmen_liste", "listen_staerke"]].groupby(["kanton_bezeichnung", "kanton_nummer", "partei_id"]).sum().reset_index()

In [10]:
## calculate number of votes for seat in parliament
## calculations based on https://www.bk.admin.ch/dam/bk/de/dokumente/pore/Leitfaden%20f%C3%BCr%20kandidierende%20Gruppierungen.pdf.download.pdf/Leitfaden%20f%C3%BCr%20kandidierende%20Gruppierungen.pdf

for kanton in tqdm(parteien.kanton_nummer.unique()):
    verteilungszahl = math.ceil(parteien[parteien.kanton_nummer == kanton].stimmen_liste.sum() / (parteien[parteien.kanton_nummer == kanton].anzahl_gewaehlte.sum() + 1))
    
    for i, row in parteien[parteien['kanton_nummer'] == kanton].iterrows():
        parteien.loc[i, "verteilungszahl"] = verteilungszahl

100%|██████████| 26/26 [00:00<00:00, 150.68it/s]


In [11]:
## calculate number of full mandate for each party
for i, row in tqdm(parteien.iterrows(), total=len(parteien)):
    parteien.loc[i, "sitze_parteien"] = math.floor(row['stimmen_liste'] / row['verteilungszahl'])
    parteien.loc[i, "reststimmen"] = row['stimmen_liste'] % row['verteilungszahl']

100%|██████████| 253/253 [00:00<00:00, 1158.84it/s]


In [19]:
## distribute remaining seats
parteien["restmandate"] = 0

for kanton in tqdm(parteien.kanton_nummer.unique()):
    restmandate = parteien[parteien["kanton_nummer"] == kanton].anzahl_gewaehlte.sum() - parteien[parteien["kanton_nummer"] == kanton].sitze_parteien.sum()
    
    while restmandate > 0:
        temp = parteien[parteien["kanton_nummer"] == kanton]
        temp["verteilungrest"] = temp['stimmen_liste'] / (temp['sitze_parteien'] + temp['restmandate'] + 1)
        idx = temp.sort_values(["verteilungrest"], ascending=False).index[0]
        parteien.loc[idx, "restmandate"] += 1
        restmandate -= 1

  0%|          | 0/26 [00:00<?, ?it/s]/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
100%|██████████| 26/26 [00:01<00:00, 19.81it/s]


In [13]:
## sum up seats
parteien['sitze_modell'] = parteien['sitze_parteien'] + parteien['restmandate']

In [14]:
## calc difference to real result
parteien["sitze_diff"] = parteien.anzahl_gewaehlte - parteien.sitze_modell

In [15]:
## look at result (new distribution)
pd.merge(parteien[((parteien.sitze_modell != 0))],
         meta_parteien[["partei_id", "partei_bezeichnung_kurz_de"]],
         on="partei_id",
         how="left")[["partei_bezeichnung_kurz_de", "sitze_modell"]].groupby("partei_bezeichnung_kurz_de").sum()

,sitze_modell
partei_bezeichnung_kurz_de,
BDP,3.0
CVP,23.0
EVP,2.0
FDP,29.0
GLP,11.0
GPS,30.0
LPS,1.0
Lega,1.0
PdA/Sol.,2.0


In [16]:
## look at differences to real result
pd.merge(parteien[parteien.sitze_diff != 0][["partei_id", "sitze_diff"]].groupby("partei_id").sum().reset_index(),
         meta_parteien[["partei_id", "partei_bezeichnung_kurz_de"]],
         on="partei_id",
         how="left")[["partei_bezeichnung_kurz_de", "sitze_diff"]]

,partei_bezeichnung_kurz_de,sitze_diff
0,FDP,-1.0
1,CVP,2.0
2,SP,1.0
3,SVP,-7.0
4,EVP,1.0
5,GPS,-2.0
6,EDU,1.0
7,GLP,5.0


In [117]:
## export differences to real result
pd.merge(parteien[parteien.sitze_diff != 0][["partei_id", "sitze_diff"]].groupby("partei_id").sum().reset_index(),
         meta_parteien[["partei_id", "partei_bezeichnung_kurz_de"]],
         on="partei_id",
         how="left")[["partei_bezeichnung_kurz_de", "sitze_diff"]].to_csv("output/diff_keine_listenverbindungen.csv", index=False)

In [17]:
## look at differences to real result on cantonal level
pd.merge(parteien[((parteien.sitze_diff != 0))],
         meta_parteien[["partei_id", "partei_bezeichnung_kurz_de"]],
         on="partei_id",
         how="left")[["kanton_bezeichnung", "kanton_nummer", "partei_bezeichnung_kurz_de", "partei_id", "anzahl_gewaehlte", "sitze_modell", "restmandate", "sitze_diff"]]

,kanton_bezeichnung,kanton_nummer,partei_bezeichnung_kurz_de,partei_id,anzahl_gewaehlte,sitze_modell,restmandate,sitze_diff
0,Aargau,19,EVP,7,1,0.0,0,1.0
1,Aargau,19,GPS,13,1,2.0,1,-1.0
2,Basel-Landschaft,13,CVP,2,1,0.0,0,1.0
3,Basel-Landschaft,13,GPS,13,1,2.0,1,-1.0
4,Basel-Stadt,12,SVP,4,0,1.0,1,-1.0
5,Basel-Stadt,12,GLP,31,1,0.0,0,1.0
6,Bern / Berne,2,SP,3,4,5.0,1,-1.0
7,Bern / Berne,2,SVP,4,7,8.0,1,-1.0
8,Bern / Berne,2,EDU,16,1,0.0,0,1.0
9,Bern / Berne,2,GLP,31,3,2.0,0,1.0


In [111]:
## find affected politicians
affected_seats = pd.merge(parteien[parteien.sitze_diff != 0][["partei_id", "kanton_bezeichnung", "kanton_nummer", "sitze_diff"]].groupby(["kanton_nummer", "kanton_bezeichnung", "partei_id"]).sum().reset_index(),
                         meta_parteien[["partei_id", "partei_bezeichnung_kurz_de"]],
                         on="partei_id",
                         how="left")

for i, row in affected_seats.iterrows():
    candidates = nr_kandidierende[(nr_kandidierende.kanton_nummer == row['kanton_nummer']) & (nr_kandidierende.kandidat_partei_id == row['partei_id'])].sort_values("stimmen_kandidat", ascending=False)
    for j, candidate in candidates.iterrows():
        if candidate['flag_gewaehlt'] == False:
            if row['sitze_diff'] == -1:
                print("{} - anstelle von: {} {} ({})".format(row['kanton_bezeichnung'], candidate['vorname'], candidate['name'], row['partei_bezeichnung_kurz_de']))
            elif row['sitze_diff'] == 1:
                print("{} - ist gewählt: {} {} ({})".format(row['kanton_bezeichnung'], candidates.loc[last_j,'vorname'], candidates.loc[last_j,'name'], row['partei_bezeichnung_kurz_de']))
            break
        last_j = j

Zürich - anstelle von: Benjamin (Beni) Fischer (SVP)
Zürich - ist gewählt: Judith Bellaïche (GLP)
Bern / Berne - anstelle von: Corrado Pardini (SP)
Bern / Berne - anstelle von: Lars Guggisberg (SVP)
Bern / Berne - ist gewählt: Andreas Gafner (EDU)
Bern / Berne - ist gewählt: Melanie Mettler (GLP)
Luzern - anstelle von: Vroni Thalmann (SVP)
Luzern - ist gewählt: Roland Fischer (GLP)
Fribourg / Freiburg - ist gewählt: Marie-France Roth Pasquier (CVP)
Fribourg / Freiburg - anstelle von: Jean-François Rime (SVP)
Basel-Stadt - anstelle von: Sebastian Frehner (SVP)
Basel-Stadt - ist gewählt: Katja Christ (GLP)
Basel-Landschaft - ist gewählt: Elisabeth Schneider-Schneiter (CVP)
Basel-Landschaft - anstelle von: Florence Brenzikofer (GPS)
Graubünden / Grigioni / Grischun - ist gewählt: Sandra Locher Benguerel (SP)
Graubünden / Grigioni / Grischun - anstelle von: Heinz Brand (SVP)
Aargau - ist gewählt: Lilian Studer (EVP)
Aargau - anstelle von: Ruth Müri (GPS)
Thurgau - anstelle von: Hansjörg Br

---

## Szenario 2
#### Sitzverteilung falls die FDP konsequent Listenverbindungen mit der SVP eingeht (Und in die bestenden Listenverbindungen der SVP eintritt)

In [38]:
# create deep copy of dataframe
svpfdp_list = copy.deepcopy(listen)

In [42]:
# listenverbindung before (example)
svpfdp_list[(svpfdp_list.kanton_nummer == 1) &
            ((svpfdp_list.partei_id == 1) |
             (svpfdp_list.partei_id == 4))
           ][["kanton_bezeichnung",
              "liste_bezeichnung",
              "liste_unterlistenverbindung",
              "liste_verbindung",
              "partei_id",
              "stimmen_liste"]]


,kanton_bezeichnung,liste_bezeichnung,liste_unterlistenverbindung,liste_verbindung,partei_id,stimmen_liste
12,Zürich,"SVP Schweizerische Volkspartei, Junge",A1,A,4,50724
18,Zürich,SVP Schweizerische Volkspartei,A1,A,4,3775767
24,Zürich,FDP.Die Liberalen,,C,1,1944944
25,Zürich,"SVP Schweizerische Volkspartei, 55plus",A1,A,4,70050
30,Zürich,Jungfreisinnige Kanton Zürich,,C,1,48700


In [43]:
# loop through cantons
for kanton in svpfdp_list.kanton_nummer.unique():
    # get list link of SVP
    svp_verbindung = svpfdp_list[(svpfdp_list.kanton_nummer == kanton) & (svpfdp_list.partei_id == 4)].liste_verbindung.unique()
    
    if len(svp_verbindung) > 0:
        
        # if SVP is not part of any Listenverbindung, create one
        if svp_verbindung[0] == ' ':
            svp_verbindung[0] = "X"
            
        for i, row in svpfdp_list[(svpfdp_list.kanton_nummer == kanton)].iterrows():
            
            # add SVP to existing (or new) Listenverbindung
            if (row['partei_id'] == 4):
                svpfdp_list.loc[i, 'liste_verbindung'] = svp_verbindung[0]
                svpfdp_list.loc[i, 'liste_unterlistenverbindung'] = " "
            
            # add FDP to the same Listenverbindung
            if (row['partei_id'] == 1):
                svpfdp_list.loc[i, 'liste_verbindung'] = svp_verbindung[0]    
                svpfdp_list.loc[i, 'liste_unterlistenverbindung'] = " "   
    

In [44]:
# listenverbindung after (example)
svpfdp_list[(svpfdp_list.kanton_nummer == 1) &
            ((svpfdp_list.partei_id == 1) |
             (svpfdp_list.partei_id == 4))
           ][["kanton_bezeichnung",
              "liste_bezeichnung",
              "liste_unterlistenverbindung",
              "liste_verbindung",
              "partei_id",
              "stimmen_liste"]]

,kanton_bezeichnung,liste_bezeichnung,liste_unterlistenverbindung,liste_verbindung,partei_id,stimmen_liste
12,Zürich,"SVP Schweizerische Volkspartei, Junge",,A,4,50724
18,Zürich,SVP Schweizerische Volkspartei,,A,4,3775767
24,Zürich,FDP.Die Liberalen,,A,1,1944944
25,Zürich,"SVP Schweizerische Volkspartei, 55plus",,A,4,70050
30,Zürich,Jungfreisinnige Kanton Zürich,,A,1,48700


In [45]:
# calculate new distribution using calc_seats()
svpfdp_listenverbindungen_res = calc_seats(svpfdp_list)

In [61]:
# look at new seat distribution
svpfdp_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,22
2,EDU,1
3,EVP,3
4,FDP,28
5,GLP,14
6,GPS,28
7,LPS,1
8,Lega,2
9,PdA/Sol.,2


In [48]:
# save new seat distribution to csv
svpfdp_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index().to_csv("output/svpfdp_listenverbindung.csv", index=False)

---

## Szenario 3
Sitzverteilung falls die Grünen keine Listenverbindungen mit der SP eingingen

In [52]:
# create deep copy of dataframe
gpssp_list = copy.deepcopy(listen)

In [53]:
# loop through cantons
for kanton in gpssp_list.kanton_nummer.unique():
    # get listenverbindung where SP is part of
    sp_verbindung = gpssp_list[(gpssp_list.kanton_nummer == kanton) & (gpssp_list.partei_id == 3)].liste_verbindung.unique()
    
    if len(sp_verbindung) > 0:
        
        if sp_verbindung[0] != ' ':
            
            for i, row in gpssp_list[(gpssp_list.kanton_nummer == kanton)].iterrows():
                
                if (row['partei_id'] == 13):
                    
                    # if GPS is part of the same Listenverbindung. move them to a new one -> X
                    if (row['liste_verbindung'] == sp_verbindung[0]):
                        gpssp_list.loc[i, 'liste_verbindung'] = 'X'
                        gpssp_list.loc[i, 'liste_unterlistenverbindung'] = ' '
    

In [54]:
# calculate new distribution using calc_seats()
spgps_listenverbindungen_res = calc_seats(gpssp_list, print_output = False)

In [59]:
# look at new seat distribution
spgps_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,27
2,EDU,1
3,EVP,3
4,FDP,29
5,GLP,16
6,GPS,25
7,LPS,1
8,Lega,1
9,PdA/Sol.,2


In [118]:
spgps_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index().to_csv("output/spgps_listenverbindung.csv", index=False)

---

## ------ weitere Szenarien ------

---

#### Szenario FDP und CVP konsequent zusammen
Sitzverteilung falls die FDP und CVP konsequent Listenverbindungen eingingen (FDP wird Teil der CVP-Listen)

In [63]:
fdpcvp_list = copy.deepcopy(listen)

In [64]:
for kanton in fdpcvp_list.kanton_nummer.unique():
    cvp_verbindung = fdpcvp_list[(fdpcvp_list.kanton_nummer == kanton) & (fdpcvp_list.partei_id == 2)].liste_verbindung.unique()
    
    if len(cvp_verbindung) > 0:
        
        if cvp_verbindung[0] == ' ':
            cvp_verbindung[0] = "X"
            
        for i, row in fdpcvp_list[(fdpcvp_list.kanton_nummer == kanton)].iterrows():
            
            if (row['partei_id'] == 2):
                fdpcvp_list.loc[i, 'liste_verbindung'] = cvp_verbindung[0]
                fdpcvp_list.loc[i, 'liste_unterlistenverbindung'] = ' '
                
            if (row['partei_id'] == 1):
                fdpcvp_list.loc[i, 'liste_verbindung'] = cvp_verbindung[0]
                fdpcvp_list.loc[i, 'liste_unterlistenverbindung'] = ' '  
    

In [65]:
fdpcvp_listenverbindungen_res = calc_seats(fdpcvp_list)

In [66]:
fdpcvp_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,23
2,EDU,1
3,EVP,3
4,FDP,35
5,GLP,16
6,GPS,28
7,LPS,1
8,Lega,1
9,PdA/Sol.,2


In [119]:
fdpcvp_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index().to_csv("output/fdpcvp_listenverbindung.csv", index=False)

---

#### Szenario keine Mitte-Listenverbindung
Sitzverteilung falls die 4 Mitte keine Listenverbindungen einginge

In [68]:
mitte_list = copy.deepcopy(listen)

In [69]:
for kanton in mitte_list.kanton_nummer.unique():
    mitte_verbindung = mitte_list[(mitte_list.kanton_nummer == kanton) & (mitte_list.partei_id == 7)].liste_verbindung.unique()
    
    if len(mitte_verbindung) > 0:
        
        if mitte_verbindung[0] != ' ':
            party_list = mitte_list[(mitte_list.kanton_nummer == kanton) & (mitte_list.liste_verbindung == mitte_verbindung[0])].partei_id.unique()
            
            if ((7 in party_list) & (2 in party_list) & (32 in party_list) & (31 in party_list)):
                print(mitte_list[mitte_list.kanton_nummer == kanton].kanton_bezeichnung.unique()[0])
                
                for i, row in mitte_list[(mitte_list.kanton_nummer == kanton)].iterrows():
                    
                    if (row['partei_id'] == 7):
                        mitte_list.loc[i, 'liste_verbindung'] = 'X'
                        mitte_list.loc[i, 'liste_unterlistenverbindung'] = ' '
                        
                    if (row['partei_id'] == 2):
                        mitte_list.loc[i, 'liste_verbindung'] = 'Y'
                        mitte_list.loc[i, 'liste_unterlistenverbindung'] = ' '
                        
                    if (row['partei_id'] == 32):
                        mitte_list.loc[i, 'liste_verbindung'] = 'Z'
                        mitte_list.loc[i, 'liste_unterlistenverbindung'] = ' '
                        
                    if (row['partei_id'] == 31):
                        mitte_list.loc[i, 'liste_verbindung'] = 'W'
                        mitte_list.loc[i, 'liste_unterlistenverbindung'] = ' '
    

Zürich
Bern / Berne
Fribourg / Freiburg
Solothurn
Basel-Stadt
Basel-Landschaft
St. Gallen


In [70]:
mitte_listenverbindungen_res = calc_seats(mitte_list)

In [71]:
mitte_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,23
2,EDU,1
3,EVP,3
4,FDP,28
5,GLP,13
6,GPS,29
7,LPS,1
8,Lega,1
9,PdA/Sol.,2


In [120]:
mitte_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index().to_csv("output/ohne_mitteallianz_listenverbindung.csv", index=False)

---

#### Szenario FDP
Sitzverteilung falls die FDP keine Listenverbindungen eingeht

In [102]:
fdp_listenverbindungen = del_lvs(1, basislisten)

In [103]:
fdp_listenverbindungen_res = calc_seats(fdp_listenverbindungen)

In [104]:
fdp_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,24
2,EDU,1
3,EVP,3
4,FDP,27
5,GLP,16
6,GPS,28
7,LPS,1
8,Lega,2
9,PdA/Sol.,2


---

#### Szenario EDU
Sitzverteilung falls die EDU keine Listenverbindungen eingeht

In [91]:
edu_listenverbindungen = del_lvs(16, listen)

In [92]:
edu_listenverbindungen_res = calc_seats(edu_listenverbindungen)

In [93]:
edu_listenverbindungen_res[["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,3
1,CVP,25
2,EVP,3
3,FDP,28
4,GLP,16
5,GPS,28
6,LPS,1
7,Lega,1
8,PdA/Sol.,2
9,SP,39


In [94]:
edu_listenverbindungen_res[edu_listenverbindungen_res.kanton_nummer == 2][["parteiname", "sitze"]].groupby(["parteiname"]).sum().reset_index()

,parteiname,sitze
0,BDP,2
1,EVP,1
2,FDP,2
3,GLP,3
4,GPS,4
5,SP,4
6,SVP,8
